1.使用线性回归模型在披萨训练样本上进行拟合

In [10]:
X_train=[[6],[8],[10],[14],[18]]
y_train=[[7],[9],[13],[17.5],[18]]

# 导入线性回归模型
from sklearn.linear_model import LinearRegression
regressor=LinearRegression()
regressor.fit(X_train,y_train)

# 导入numpy
import numpy as np
# 在x轴上从0-25均匀采样100个数据点,并以100个数据点为基准，预测回归直线
xx=np.linspace(0,26,100)
xx=xx.reshape(xx.shape[0],1)
yy=regressor.predict(xx)

# 对预测到的直线作图,
import matplotlib.pyplot as plt
plt.scatter(X_train,y_train)
# 使用plt.plot()画(x,y)曲线,degree=1表示特征是一维的，做个标记
plt1,=plt.plot(xx,yy,label="Degree=1")
plt.axis([0,25,0,25]) # axis表示坐标的极值范围
plt.xlabel('Diameter of pizza')
plt.ylabel('Price') 
plt.show()

# 输出模型在训练样本上的R-squared值
print regressor.score(X_train,y_train)

0.9100015964240102


接下来我们尝试将原特征提高一个维度，用2次多项式回归来拟合训练样本

2.使用2次多项式回归模型在训练样本上进行拟合

In [15]:
from sklearn.preprocessing import PolynomialFeatures
# 使用PolynomialFeatures(degree=2)映射出2次多项式特征
poly2=PolynomialFeatures(degree=2)
X_train_poly2=poly2.fit_transform(X_train)

# 以线性回归模型为基础，初始化模型（特征维度提升，但模型仍是线性回归模型）
regressor_poly2=LinearRegression()
regressor_poly2.fit(X_train_poly2,y_train)

# 从新映射绘图用x轴采样数据
xx_poly2=poly2.transform(xx)

# 预测
yy_poly2=regressor_poly2.predict(xx_poly2)

# 作图
plt.scatter(X_train,y_train)
plt1,=plt.plot(xx,yy,label='degree=1')
plt2,=plt.plot(xx,yy_poly2,label='degree=2')
plt.axis([0,25,0,25])
plt.xlabel('diameter')
plt.ylabel('price')
plt.show()

print regressor_poly2.score(X_train_poly2,y_train)

0.9816421639597428


果然在升高特征维度后，模型性能更高，对训练数据的拟合程度更好。接下来我们进一步提高特征维度。

In [16]:
from sklearn.preprocessing import PolynomialFeatures
# 使用PolynomialFeatures(degree=4)映射出2次多项式特征
poly4=PolynomialFeatures(degree=4)
X_train_poly4=poly4.fit_transform(X_train)

# 以线性回归模型为基础，初始化模型（特征维度提升，但模型仍是线性回归模型）
regressor_poly4=LinearRegression()
regressor_poly4.fit(X_train_poly4,y_train)

# 从新映射绘图用x轴采样数据
xx_poly4=poly4.transform(xx)

# 预测
yy_poly4=regressor_poly4.predict(xx_poly4)

# 作图
plt.scatter(X_train,y_train)
plt1,=plt.plot(xx,yy,label='degree=1')
plt2,=plt.plot(xx,yy_poly2,label='degree=2')
plt3,=plt.plot(xx,yy_poly4,label='degree=4')
plt.axis([0,25,0,25])
plt.xlabel('diameter')
plt.ylabel('price')
plt.show()

print regressor_poly4.score(X_train_poly4,y_train)

1.0


由图和R平方指标可见，4次多项式曲线几乎完全拟合了所有训练样本点。接下来我们看着三种特征维度下的模型分别在测试集上的性能表现。

3.评估3种回归模型在测试集上的性能表现

In [18]:
X_test=[[6],[8],[11],[16]]
y_test=[[8],[12],[15],[18]]

In [19]:
# degree=1
print regressor.score(X_test,y_test)

# degree=2
X_test_poly2=poly2.transform(X_test)
print regressor_poly2.score(X_test_poly2,y_test)

# degree=4
X_test_poly4=poly4.transform(X_test)
print regressor_poly4.score(X_test_poly4,y_test)

0.809726797707665
0.8675443656345108
0.8095880795788558


由输出可见

- 欠拟合：当模型复杂度很低时(degree=1)，模型既在训练集上拟合不好，又在测试集上表现一般
- 过拟合：一味追求高模型复杂度(degree=4)，尽管模型完美拟合了几乎所有训练数据，但模型会变得非常波动，几乎丧失了对未知数据的预测能力

这两种都是模型缺乏泛化力的表现。

要求我们在增加模型复杂度、提高在可观测数据上的性能表现的同时，需要兼顾模型的泛化力，防止发生过拟合。为了平衡这两种选择，我们通常采用2种模型正则化方法：L1范数正则化 & L2范数正则化 

2.Lasso模型在4次多项式特征上的拟合表现

In [23]:
from sklearn.linear_model import Lasso
lasso_poly4=Lasso()
lasso_poly4.fit(X_train_poly4,y_train)
print lasso_poly4.score(X_test_poly4,y_test)
# 输出lasso模型的参数列表
print lasso_poly4.coef_

0.8388926873604382
[ 0.00000000e+00  0.00000000e+00  1.17900534e-01  5.42646770e-05
 -2.23027128e-04]


In [22]:
# 回顾普通4次多项式回归模型拟合后的性能和参数列表
print regressor_poly4.score(X_test_poly4,y_test)
print regressor_poly4.coef_

0.8095880795788558
[[ 0.00000000e+00 -2.51739583e+01  3.68906250e+00 -2.12760417e-01
   4.29687500e-03]]


由上可见，默认配置的lasso模型性能提高了约3%。lasso模型拟合后的参数列表中，4次与3次特征的参数均为0.0，使得特征更加稀疏。

3.Ridge模型在4次多项式特征上的拟合表现

In [25]:
# 输出普通4次多项式回归模型的参数列表
print regressor_poly4.coef_
# 输出上述参数的平方和，验证参数间的巨大差异
print np.sum(regressor_poly4.coef_ **2)

[[ 0.00000000e+00 -2.51739583e+01  3.68906250e+00 -2.12760417e-01
   4.29687500e-03]]
647.3826457370965


In [27]:
from sklearn.linear_model import Ridge
ridge_poly4=Ridge()
ridge_poly4.fit(X_train_poly4,y_train)
print ridge_poly4.score(X_test_poly4,y_test)
print ridge_poly4.coef_
print np.sum(ridge_poly4.coef_ **2)

0.8374201759366331
[[ 0.         -0.00492536  0.12439632 -0.00046471 -0.00021205]]
0.01549896520353474


由输出可见，相比普通4次多项式回归模型，默认配置下的Ridge模型性能提高了约3%，且模型拟合后的参数间差异非常小。

λ是调节因子。